In [1]:
!pip install pandarallel
import pandas as pd
pd.set_option("max_colwidth", 600)
from bs4 import BeautifulSoup
import requests

from pandarallel import pandarallel


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandarallel: filename=pandarallel-1.6.2-py3-none-any.whl size=16411 sha256=e8c25063320bab8386d9886447f1f17db778fb10f669161fd317b56de0fcb6e3
  Stored in directory: /root/.cache/pip/wheels/01/8d/99/4ce6a1e914ed6ec383bc099dfbe0112dfde0bc39957ca93ec1
Successfully built pandarallel


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def extract_urls_topic(url):
    try:
      coverpage = requests.get(url).content
      soup = BeautifulSoup(coverpage, 'html.parser')
     
      rows = []
      coverpage=soup.find_all('loc')
      for link in coverpage:
        if link.get_text().find('internacional')>=0 :
          rows.append(link.get_text()) 
        else :
          if link.get_text().find('espana')>=0:
            rows.append(link.get_text()) 
    except:
      rows = 'error'
      
    return rows

In [9]:
extract_urls_topic('https://www.elperiodicoextremadura.com/sitemap_contents_2021_11_d9fc2_001.xml')

['https://www.elperiodicoextremadura.com/espana/2021/11/30/fiscalia-ceuta-archiva-devolucion-caliente-60172438.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/sanidad-recomienda-limitar-numero-personas-60167545.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/netflix-hbo-tendran-cumplir-cuota-60156819.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/erc-amenaza-votar-presupuestos-cuota-60169296.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/incidencia-covid-supera-200-notificar-60166545.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/almudena-grandes-sera-hija-predilecta-60171863.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/crisis-barbara-rey-provoco-tombola-60171794.html',
 'https://www.elperiodicoextremadura.com/internacional/2021/11/30/tiroteo-instituto-michigan-deja-tres-60170439.html',
 'https://www.elperiodicoextremadura.com/espana/2021/11/30/pp-inicia-proceso-interno-cay

In [10]:
pandarallel.initialize(nb_workers= 30, progress_bar=True)

url_str=''

try:
  coverpage = requests.get('https://www.elperiodicoextremadura.com/sitemap_index_d9fc2.xml').content
  soup = BeautifulSoup(coverpage, 'html.parser')
  
  rows =[]
  coverpage=soup.find_all('loc')
  for link in coverpage:
    rows.append(link.get_text()) 
except:
  rows = 'error'
df = pd.DataFrame(range(0,34),columns = ['page'])
df['page']=rows
print(df)
df['rows']=df['page'].parallel_apply(extract_urls_topic)
df = df[df.rows.str.len()!=0].reset_index(drop=True)
df = df.explode('rows', ignore_index=True)
df=df.drop_duplicates(subset='rows')
df = df.dropna()
print(df)
df.to_csv('/content/drive/MyDrive/Colab Notebooks/elperiodicoextremadura/url.csv',index=False)

INFO: Pandarallel will run on 30 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
                                                                                page
0      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml
1               https://www.elperiodicoextremadura.com/sitemap_google_news_d9fc2.xml
2                     https://www.elperiodicoextremadura.com/sitemap_today_d9fc2.xml
3             https://www.elperiodicoextremadura.com/sitemap_community_d9fc2_001.xml
4                  https://www.elperiodicoextremadura.com/sitemap_city_d9fc2_001.xml
5                  https://www.elperiodicoextremadura.com/sitemap_area_d9fc2_001.xml
6                   https://www.elperiodicoextremadura.com/sitemap_tag_d9fc2_002.xml
7                   https://www.elperiodicoextremadura.com/sitemap_tag_d9fc2_001.xml
8               https://www.elperiodicoextremadura.com/sitemap_section_d9fc2_001.xml
9      

                                                                                page  \
0      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml   
1      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml   
2      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml   
3      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml   
4      https://www.elperiodicoextremadura.com/sitemap_contents_2022_08_d9fc2_001.xml   
...                                                                              ...   
20325  https://www.elperiodicoextremadura.com/sitemap_contents_2022_03_d9fc2_001.xml   
20327  https://www.elperiodicoextremadura.com/sitemap_contents_2022_03_d9fc2_001.xml   
20328  https://www.elperiodicoextremadura.com/sitemap_contents_2022_03_d9fc2_001.xml   
20329  https://www.elperiodicoextremadura.com/sitemap_contents_2022_03_d9fc2_001.xml   
20330  https://www.elperiodicoex